In [1]:
import sys
from pathlib import Path

# Add src/ to Python path
project_root = Path.cwd().parent  # assuming notebook is inside notebooks/
src_path = project_root / "src"
sys.path.insert(0, str(src_path))

# Now you can import your module
from shared.gridded_data_sources.cetb import CETBScene
from shared.utils.grid_utils import Grid

In [ ]:
scene = CETBScene(r"C:\Users\ehate\Desktop\ASP\ASP_code_projects\raw_data\CETB_AMSR2\2023-2024\NSIDC0630_SIR_EASE2_N3.125km_GCOMW1_AMSR2_M_36H_20231101_2409031549_v2.0.nc").load()

target_grid = Grid.from_predefined('EASE2_N6.25km')

In [4]:
grid1 = Grid.from_predefined(grid_id = 'EASE2_N25km')
grid2 = Grid.from_predefined(grid_id = 'EASE2_N12.5km')

grid1 == grid2

False

In [5]:
test = grid1.create_grid()
test

<xarray.Dataset> Size: 12kB
Dimensions:  (x: 720, y: 720)
Coordinates:
  * x        (x) float64 6kB -8.988e+06 -8.962e+06 ... 8.962e+06 8.988e+06
  * y        (y) float64 6kB 8.988e+06 8.962e+06 ... -8.962e+06 -8.988e+06
Data variables:
    *empty*

In [9]:
scene = CETBScene(
    [
    r"C:\Users\ehate\Desktop\ASP\ASP_code_projects\raw_data\CETB_AMSR2\2023-2024\NSIDC0630_SIR_EASE2_N3.125km_GCOMW1_AMSR2_M_36H_20231101_2409031549_v2.0.nc",
    r"C:\Users\ehate\Desktop\ASP\ASP_code_projects\raw_data\CETB_AMSR2\2023-2024\NSIDC0630_SIR_EASE2_N3.125km_GCOMW1_AMSR2_M_36H_20231102_2409031549_v2.0.nc"
    ]
    )
scene.load()
#ds

In [13]:
scene.data = scene.data.rename({'TB': 'test'})
scene.data

<xarray.Dataset> Size: 2GB
Dimensions:          (time: 2, y: 5760, x: 5760)
Coordinates:
  * time             (time) datetime64[ns] 16B 2023-11-01 2023-11-02
  * y                (y) float64 46kB 8.998e+06 8.995e+06 ... -8.998e+06
  * x                (x) float64 46kB -8.998e+06 -8.995e+06 ... 8.998e+06
Data variables:
    crs              |S1 1B b''
    test             (time, y, x) float32 265MB dask.array<chunksize=(1, 1440, 1440), meta=np.ndarray>
    TB_num_samples   (time, y, x) float32 265MB dask.array<chunksize=(1, 1920, 1920), meta=np.ndarray>
    Incidence_angle  (time, y, x) float32 265MB dask.array<chunksize=(1, 1440, 1440), meta=np.ndarray>
    TB_std_dev       (time, y, x) float32 265MB dask.array<chunksize=(1, 1440, 1440), meta=np.ndarray>
    TB_time          (time, y, x) datetime64[ns] 531MB dask.array<chunksize=(1, 1440, 1440), meta=np.ndarray>
Attributes: (12/76)
    references:                    ['Long, D. G. and M. J. Brodzik. 2016. Opt...
    title:                         Calibrated Enhanced-Resolution Passive Mic...
    id:                            10.5067/19LHYLUXZ22M
    summary:                       This data set is an enhanced-resolution, g...
    project:                       ['NASA MEaSUREs: An improved, enhanced-res...
    contributor_name:              Mary J. Brodzik, David G. Long, Molly A. H...
    ...                            ...
    input_file15:                  1C.GCOMW1.AMSR2.XCAL2016-V.20231101-S12212...
    input_file16:                  1C.GCOMW1.AMSR2.XCAL2016-V.20231101-S14001...
    input_file17:                  1C.GCOMW1.AMSR2.XCAL2016-V.20231101-S15390...
    input_file18:                  1C.GCOMW1.AMSR2.XCAL2016-V.20231101-S17180...
    input_file19:                  1C.GCOMW1.AMSR2.XCAL2016-V.20231101-S18565...
    comment:                       Epoch date for data in this file: 2023-11-...